In [241]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import json
import datetime
import re

In [242]:
df_scrap = pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag']  )
df_scrap

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag


In [243]:
""" Works for both 2 urls """

#url = "https://www.data.gouv.fr/fr/datasets/les-elus-municipaux/"
url = "https://www.data.gouv.fr/fr/datasets/repertoire-national-des-elus-1/"


In [244]:
def getSoup(url):
    
    response = requests.get(url)
    print("statut de la reponse ", response.status_code)
    print('\n')
    soup = bs(response.text, 'html.parser')
    print(soup)
    
    return soup

In [245]:
soup = getSoup(url)

statut de la reponse  200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta charset="utf-8"/>
<title>Répertoire national des élus (RNE) - data.gouv.fr</title>
<meta content="website" property="og:type"/>
<meta content="Répertoire national des élus (RNE) - data.gouv.fr" property="og:title"/>
<link href="http://www.etalab.gouv.fr/" rel="author"/>
<meta content="Le Répertoire National des Elus (RNE) a pour finalité le…" name="description"/>
<meta content="Le Répertoire National des Elus (RNE) a pour finalité le…" property="og:description"/>
<meta content="https://static.data.gouv.fr/avatars/8e/0925ee5c44429d9e37131258773cb7-100.png" property="og:image"/>
<meta content="opendata, udata, jeu de données, elections, elections-cantonales, elections-europeennes, elections-legislatives, elections-municipales, elections-regionales, rne" name="keywords"/>
<meta content="https://static.data.gouv.fr/_themes/gouvfr/" name="theme-static-root"/>
<meta content="https://static.data.gouv.fr/static/" name="s

In [246]:
def getContent(soup):
    
    """ 
        Extracting the content of the article, whether in html or not
    """
    if soup.find_all('section', class_="content noncertified") == [] :
        art_content_html = soup.find_all('section', class_="content certified")
    else :
        art_content_html = soup.find_all('section', class_="content noncertified")

    art_content = " ".join([x.text for x in art_content_html])

        #art_content = art_content.replace("\n", "").replace("\xa0", "")

    return art_content, art_content_html

In [247]:
def getDate(soup):
    
    """ 
        Extracting the date of publishing / updating of the article
    """
    
    art_extract_datetime = datetime.date.today()
    
    return art_extract_datetime

In [248]:
getDate(soup)

datetime.date(2021, 1, 7)

In [249]:
def getLang(soup):
    
    """ 
        Extracting the language the article is written in
    """
    
    art_lang = soup.find('html').get('lang')

    return art_lang

In [250]:
def getTitle(soup):
    
    """ 
        Extracting the title of the article
    """
    
    art_title = soup.find('meta', property = "og:title").get('content')
    
    return art_title

In [251]:
def getUrl(soup):
    
    """ 
        Extracting the url of the article
    """
    
    art_url = soup.find('link', rel = 'canonical').get('href')
    
    return art_url

In [252]:
def getSourceName(soup):
    
    """ 
        Extracting article's source name
    """
    
    #src_name = soup.find('meta', attrs={'name':"site-title"}).get('content')
    src_name = getSourceUrl(soup).replace('https://','').replace('/','')
    
    return src_name

In [253]:
def getSourceType(soup):
    
    """ 
        Extracting article's source type
    """
    
    src_type = "xpath_source"
    
    return src_type

In [254]:
def getSourceUrl(soup):
    
    """ 
        Article's source url
    """
    
    #src_url = soup.find('meta', attrs={'name':"site-title"}).get('content')
    #return src_url
    
    for val in re.finditer("(\w)+://[^/]+/", url):
        return val.group(0)

In [255]:
def getImg(soup):
    
    """ 
        Extracting article's image
    """
    
    src_img = soup.find('meta', property = "og:image").get('content')
    
    return src_img

In [256]:
def getArtAuth(soup):
    
    """ 
        Extracting article's author
    """
    
    art_auth = soup.find('link', rel='author').get('href')
    
    return art_auth

In [257]:
def getTag(soup):
    
    """ 
        Extracting article's tags if they exist
    """
    
    art_tag = []
    tags = soup.find_all('a', class_ = "label label-default")
    for x in tags:
        art_tag.append(x.get('title'))
    
    return art_tag

In [258]:
def scrapPage(soup):
    
    """ 
       Function combining the previous ones and stocking the list of results in a dataframe line
    """
    
    art_content, art_content_html = getContent(soup)
    
    art_extract_datetime = getDate(soup)
    
    art_lang = getLang(soup)
    
    art_title = getTitle(soup)
    
    art_url = getUrl(soup)
    
    src_name = getSourceName(soup)
    
    src_type = "xpath_source"
    
    src_url = getSourceUrl(soup)
    
    src_img = getImg(soup) 
    
    art_auth = getArtAuth(soup)
    
    art_tag = getTag(soup)
    
    return art_content, art_content_html, art_extract_datetime, art_lang, art_title, art_url, src_name, src_type, src_url, src_img, art_auth, art_tag

In [259]:
scrapPage(soup)[5]

'/fr/datasets/repertoire-national-des-elus-1/'

In [260]:
df_scrap.loc[1]=scrapPage(soup)

In [261]:
df_scrap

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
1,\n\n\n\n\nRépertoire national des élus\n ...,"[[\n, [\n, <div class=""row"">\n<div class=""col-...",2021-01-07,fr,Répertoire national des élus (RNE) - data.gouv.fr,/fr/datasets/repertoire-national-des-elus-1/,www.data.gouv.fr,xpath_source,https://www.data.gouv.fr/,https://static.data.gouv.fr/avatars/8e/0925ee5...,http://www.etalab.gouv.fr/,"[elections, elections-cantonales, elections-eu..."
